# High performance computing

## Topics
- Why Julia is fast
- LLVM compiler
- Parallellization

## Why is Julia fast?
- Rich type information, provided naturally by multiple dispatch
- Aggressive code specialization against run-time types
- JIT compilation using the LLVM compiler framework

In short, Julia is designated from the beginning to be fast. Not vice versa.

See the [scientific paper](https://arxiv.org/pdf/1209.5145v1.pdf) behind Julia, if you want to learn more.

## Levels of parallellism
1. Instruction level parallellism
2. Vector instructions (see `Bonus_simd-vectorization.pynb` if you are interested)
3. **Threading** (shared-memory)
4. **Distributed**
5. Accelerators (e.g., GPGPU; *not covered here*)

## Advanced: A (very short) introduction to the interiors of Julia compiler

Let's stop treating our tools like blackboxes. Let's see what the compiler herself is thinking about our code.

1. `@code_lowered`
2. `@code_typed` and `@code_warntype`
3. `@code_llvm`
4. `@code_native`

See [slides](https://slides.com/valentinchuravy/julia-parallelism) by Valentin Churavy, for more.

In [9]:
function mysum(A)
    acc = zero(eltype(A))
    for a in A
        acc += a
    end
    return acc
end

mysum (generic function with 1 method)

In [10]:
@code_lowered mysum(ones(1))

CodeInfo(:(begin 
        nothing
        acc = (Main.zero)((Main.eltype)(A)) # line 3:
        SSAValue(0) = A
        #temp# = (Base.start)(SSAValue(0))
        6: 
        unless !((Base.done)(SSAValue(0), #temp#)) goto 15
        SSAValue(1) = (Base.next)(SSAValue(0), #temp#)
        a = (Core.getfield)(SSAValue(1), 1)
        #temp# = (Core.getfield)(SSAValue(1), 2) # line 4:
        acc = acc + a
        13: 
        goto 6
        15:  # line 6:
        return acc
    end))

In [11]:
@code_typed mysum(ones(1))

CodeInfo(:(begin 
        acc = (Base.sitofp)(Float64, 0)::Float64 # line 3:
        #temp# = 1
        4: 
        unless (Base.not_int)((#temp# === (Base.add_int)((Base.arraylen)(A)::Int64, 1)::Int64)::Bool)::Bool goto 14
        SSAValue(2) = (Base.arrayref)(A, #temp#)::Float64
        SSAValue(3) = (Base.add_int)(#temp#, 1)::Int64
        a = SSAValue(2)
        #temp# = SSAValue(3) # line 4:
        acc = (Base.add_float)(acc, a)::Float64
        12: 
        goto 4
        14:  # line 6:
        return acc
    end))=>Float64

In [12]:
@code_llvm mysum(ones(1))


define double @julia_mysum_63268(i8** dereferenceable(40)) #0 !dbg !5 {
top:
  %1 = getelementptr inbounds i8*, i8** %0, i64 1
  %2 = bitcast i8** %1 to i64*
  %3 = load i64, i64* %2, align 8
  %4 = icmp eq i64 %3, 0
  br i1 %4, label %L14, label %if.lr.ph

if.lr.ph:                                         ; preds = %top
  %5 = getelementptr i8*, i8** %0, i64 3
  %6 = bitcast i8** %5 to i64*
  %7 = load i64, i64* %6, align 8
  %8 = bitcast i8** %0 to double**
  %9 = load double*, double** %8, align 8
  br label %if

if:                                               ; preds = %if.lr.ph, %idxend
  %acc.06 = phi double [ 0.000000e+00, %if.lr.ph ], [ %16, %idxend ]
  %"#temp#.05" = phi i64 [ 1, %if.lr.ph ], [ %15, %idxend ]
  %10 = add i64 %"#temp#.05", -1
  %11 = icmp ult i64 %10, %7
  br i1 %11, label %idxend, label %oob

L14.loopexit:                                     ; preds = %idxend
  br label %L14

L14:                                              ; preds = %L14.loopexit, %top
  

In [13]:
@code_native mysum(ones(1))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[9]
Source line: 3
	movq	8(%rdi), %rax
	xorpd	%xmm0, %xmm0
	testq	%rax, %rax
	je	L50
	movq	(%rdi), %rdx
	movq	24(%rdi), %rsi
	xorpd	%xmm0, %xmm0
	xorl	%ecx, %ecx
	nopw	(%rax,%rax)
L32:
	cmpq	%rsi, %rcx
	jae	L51
Source line: 4
	addsd	(%rdx,%rcx,8), %xmm0
Source line: 3
	incq	%rcx
	cmpq	%rcx, %rax
	jne	L32
Source line: 6
L50:
	retq
L51:
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	movq	%rsp, %rax
	leaq	-16(%rax), %rsi
	movq	%rsi, %rsp
	incq	%rcx
	movq	%rcx, -16(%rax)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax
	nopl	(%rax)


## About global scope
So what does the previous machine code actually mean? Well, in practise:

A global variable might have its value, and therefore its type, change at any given point. This makes it difficult/nigh impossible for the compiler to reason about/optimize code using global variables.

Julia uses functions as its compilation unit and any code that is performance critical or being benchmarked should be inside a function.

## Demo: Giving hints to the compiler 
Let's see what we can do with our previously defined `laplacian` function. 

For some aggressive cases we can provide the JIT-compiler `@inbounds` macro hinting that there is no need to check array bounds.

In [5]:
#Lets re-define our previous Laplacians (from 05_ notebook)
function laplacian_bad(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ir = 2:nr-1, ic = 2:nc-1 # bad loop nesting order
        lap_x[ir,ic] =
            (x[ir+1,ic] + x[ir-1,ic] +
            x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
    end
end

#In this version, the two loops are nested properly:
function laplacian_good(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1, ir = 2:nr-1 # good loop nesting order
        lap_x[ir,ic] =
            (x[ir+1,ic] + x[ir-1,ic] +
            x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
    end
end

laplacian_good (generic function with 1 method)

In [2]:
# A way to increase the speed is to remove the array bounds checking, using the macro @inbounds:
function laplacian_good_nocheck(lap_x::Array{Float64,2}, x::Array{Float64,2})
    nr,nc = size(x)
    for ic = 2:nc-1
        for ir = 2:nr-1 # good loop nesting order
            @inbounds begin lap_x[ir,ic] = # no array bounds checking
                (x[ir+1,ic] +  x[ir-1,ic] +
                x[ir,ic+1] + x[ir,ic-1]) - 4*x[ir,ic]
            end
        end
    end
end

laplacian_good_nocheck (generic function with 1 method)

In [4]:
function main_test(nr, nc)
    field = zeros(nr, nc)
    for ic = 1:nc, ir = 1:nr
        if ir == 1 || ic == 1 || ir == nr || ic == nc
            field[ir,ic] = 1.0
        end
    end
    lap_field = zeros(size(field))

    time = @elapsed laplacian_bad(lap_field, field)
    @printf "laplacian_bad:          %.3f s\n" time
    
    time = @elapsed laplacian_good(lap_field, field)
    @printf "laplacian_good:         %.3f s\n" time
    
    time = @elapsed laplacian_good_nocheck(lap_field, field)
    @printf "laplacian_good_nocheck: %.3f s\n" time
end

main_test(10^4, 10^4)

laplacian_bad:          2.758 s
laplacian_good:         0.233 s
laplacian_good_nockeck: 0.150 s


## Threading (experimental)
Julia threading model is based on a fork-join approach and is still considered experimental.

Fork-join describes the control flow that a group of threads undergoes. Execution is then forked and an anonymous function is ran across all threads.

All threads have to join together and serial execution continues.

## Threading in practise
The number of threads Julia starts up with is controlled by an environment variable called `JULIA_NUM_THREADS`. Now, let's start up Julia with 4 threads:

```bash
export JULIA_NUM_THREADS=4
julia
```

NOTE: this does not work in the notebook environment because the kernel is automatically loaded with only 1 thread.

In [ ]:
using Base.Threads
nthreads()

## Using threads

```julia
@threads for id in 1:nthreads()
    #each thread does something
end
```

In [7]:
a = zeros(10)
@threads for i = 1:10
    a[i] = Threads.threadid()
end

## Advanced: Threaded sum
Here is a more complex example of threaded sum from https://github.com/stevengj/18S096/blob/master/lectures/lecture5/Parallelism.ipynb

In [8]:
function threaded_sum(arr)
   @assert length(arr) % nthreads() == 0
    
   let results = zeros(eltype(arr), nthreads())
       @threads for tid in 1:nthreads()
           # split work
           acc = zero(eltype(arr))
           len = div(length(arr), nthreads())
           domain = ((tid-1)*len +1):tid*len
           @inbounds for i in domain
               acc += arr[i]    
           end
           results[tid] = acc
       end
       sum(results)
   end
end

threaded_sum (generic function with 1 method)

## Distributed computing 
Distributed processing uses individual processes that communicate with each other. In this case, data movement and communication is explicit!

Julia supports various forms of distributed computing. 
- **A native master-worker system based on remote procedure calls**
- MPI through [MPI.jl](https://github.com/JuliaParallel/MPI.jl)
- [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl)

## Master-Worker model
NOTE: this does not work in the notebook environment. Instead, we need to launch Julia with 
```bash
julia -p 4
```
then inside Julia you can check
```julia
nprocs()
workers()
```
which should print `5` and `[2,3,4,5]`. 

Why 5, you ask? Because *"worker 1"* is the *"boss"*. And bosses don't work.

Functions (and everything used by workers) needs to be explicitly declared for all:
```julia
@everywhere g(x) = 2x
```
Only then can we send the job to somebody else to execute
```julia
remotecall_fetch(g, 3, 2.0)
```
Here we fetch the result of `g` of worker `3` applied to a value of `2.0`.

Use `@everywhere` to execute a top-level block on each process
```julia
@everywhere begin
    using Test
    include("src.jl")
end
```
Define variables on all processes
```julia
@everywhere bar = 1
```


## `@parallel` as a shortcut 
A parallel for loop of the form :
```julia
@parallel [reducer] for var = range
    body
end
```
The specified range is partitioned and locally executed across all workers. In case an optional reducer function is specified, @parallel performs local reductions on each worker with a final reduction on the calling process.

Note that without a reducer function, `@parallel` executes asynchronously, i.e., it spawns independent tasks on all available workers and returns immediately without waiting for completion. To wait for completion, prefix the call with `@sync`, like :
```julia
@sync @parallel for var = range
      body
end
```

In [ ]:
nheads = @parallel (+) for i=1:200000000
  Int(rand(Bool))
end

## @pmap for unbalanced load
In some cases no reduction operator is needed, and we merely wish to apply a function to all integers in some range. This is another useful operation called parallel map. 

For example, we could compute the singular values of several large random matrices in parallel as follows:

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i=1:10]
pmap(svd, M)

`pmap()` is designed for the case where each function call does a large amount of work. In contrast, `@parallel for` can handle situations where each iteration is tiny, perhaps merely summing two numbers.

## Summary: General optimization tricks

- write functions!
- Avoid global variables
    - A global variable might have its value, (and type) change at any given point. This makes it hard for the compiler to optimize.